In [52]:
# 获取data文件夹下的所有文件xlsx文件
import glob
# 获取当前完整路径
import os
path=os.getcwd() # 获取当前工作目录
path
file_names=glob.glob(f'{path}\\data\\*.xlsx')
# file_names=['data\\'+i for i in file_names]
file_names


['c:\\Users\\Administrator\\Desktop\\new_place\\WarrantyPartAnalyze-master\\data\\TABLE6_2022-04-26 09_30_46.xlsx',
 'c:\\Users\\Administrator\\Desktop\\new_place\\WarrantyPartAnalyze-master\\data\\TABLE6_2022-04-27 09_30_46 .xlsx']

In [53]:
# 加载数据
import pandas as pd
import numpy as np
# concat函数用于拼接数据
src_data=[]
for i in file_names:
    data=pd.read_excel(i)
    data['文件名']=i.replace(f'{path}\\data\\','') # 添加文件名列
    src_data.append(data) # 将每个文件的数据拼接到一起
    
df_src=pd.concat(src_data) # 将拼接好的数据放到一个dataframe中

df_src.head()

,Dealer No.,Dealer Name,Parts Number,Description,QTY,Reserve QTY,Reorder QTY,Discount,Pc,Alois Class,文件名
0,20537,Dalian Yandebao,16 13 7 404 081,通风管 带滤尘器,1,1,1,21,74,A/A6,TABLE6_2022-04-26 09_30_46.xlsx
0,20537,Dalian Yandebao,16 13 7 404 081,通风管 带滤尘器,0,1,1,21,74,A/A6,TABLE6_2022-04-27 09_30_46 .xlsx


In [54]:
df_src['Parts Number'].unique()

array(['16 13 7 404 081'], dtype=object)

In [55]:
# 处理数据
df=df_src.copy()
# 提取文件名中的时间
import datetime
# 提取时间
df['时间']=df['文件名'].apply(lambda x:datetime.datetime.strptime(x.split(' ')[0],'TABLE6_%Y-%m-%d'))
# 按经销商-时间排序-升序
df=df.sort_values(by=['Dealer No.','时间'])
df.index=df['时间']
df[['Dealer No.','QTY','Reorder QTY','时间']].head()


,Dealer No.,QTY,Reorder QTY,时间
时间,,,,
2022-04-26,20537,1,1,2022-04-26
2022-04-27,20537,0,1,2022-04-27


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2 entries, 2022-04-26 to 2022-04-27
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Dealer No.    2 non-null      int64         
 1   Dealer Name   2 non-null      object        
 2   Parts Number  2 non-null      object        
 3   Description   2 non-null      object        
 4   QTY           2 non-null      int64         
 5   Reserve QTY   2 non-null      int64         
 6   Reorder QTY   2 non-null      int64         
 7   Discount      2 non-null      int64         
 8   Pc            2 non-null      int64         
 9   Alois Class   2 non-null      object        
 10  文件名           2 non-null      object        
 11  时间            2 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int64(6), object(5)
memory usage: 208.0+ bytes


In [57]:
# groupby数据并分析
# df.groupby('Dealer No.')[23403].apply(lambda x:x['QTY']+x['Reorder QTY'])
# df.groupby('Dealer No.').get_group(23403)
# df.shape[0]
#? df['Dealer No.'].shift(-1)

# 创建函数，lambda x:x['QTY']+x['Reorder QTY']-x['QTY'].shift(1)-x['Reorder QTY'].shift(1)
def get_diff(x):
    # 
    c1=x['QTY'].shift(1)+x['Reorder QTY'].shift(1)-x['QTY']-x['Reorder QTY']
    c1=c1.map(lambda x:x if x>0 else 0) # 如果x<0，则返回0，原因是0的时候，不能减去
    # print(c1)
    
    return c1 
    

# df下一行(QTY+Reorder QTY)减去df上一行(QTY+Reorder QTY)
res_all=df.groupby(['Dealer No.','Dealer Name','Parts Number']).apply(get_diff)
# 添加汇总列
res_all['汇总']=res_all.sum(axis=1) #! 单一数据可使用

# 转换列名称为日期2022-4-28 00:00:00 为2022-4-28
res_all.columns=res_all.columns.map(lambda x:str(x)[:10]) #! 单一数据可使用

res_all



,,时间,2022-04-26,2022-04-27,汇总
Dealer No.,Dealer Name,Parts Number,,,
20537,Dalian Yandebao,16 13 7 404 081,0.0,1.0,1.0


In [59]:
# 生成报表
# todo 修正了输出逻辑
import datetime
part_num=df['Parts Number'].unique()[0].replace(' ','')
res_all.to_excel(f'{path}/output/PartNum_{part_num}-{datetime.datetime.now().strftime("%Y-%m-%d")}.xlsx')